In [2]:
import json
from fastai.text.all import get_text_files
from collections import Counter



In [3]:
import pandas as pd

In [3]:
example = json.load(open('/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/log2021-07-13 04:45:32.951661.txt'))

In [6]:
example.keys()

dict_keys(['key', 'userText', 'suggestionText', 'selectionStart', 'selectionEnd', 'acceptedSuggestion', 'duration', 'isArc'])

In [4]:
files = get_text_files('/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final')

In [4]:
def get_tabs(df):
    tab_count = 0
    for i in range(1, len(df['key'])):
        if df['key'][i] == 9 or (((len(df['userText'][i]) - len(df['userText'][i-1])) > 1) and (df['key'][i] == 39)):
            tab_count+=1
    return tab_count

In [14]:
get_tabs(example)

2

In [5]:
def mean(ls):
    if len(ls) == 0:
        return 0
    return (sum(ls)/len(ls))

In [6]:
def count_word_completes(df):
    '''
        Counts Number of times word complete was used
        Word complete happens when a user presses space or tab, the character count changes but the word count does not change 
    '''
    itemLength = len(df['key'])
    word_complete_count = 0
    for current_idx in range(1, itemLength):
         
        if df['key'][current_idx] in [9, 39]:
            if len(df['userText'][current_idx]) > len(df['userText'][current_idx-1]):
                if (len(df['userText'][current_idx].split()) == len(df['userText'][current_idx-1].split())):
                        word_complete_count += 1
               
               
    return word_complete_count
                
    
    
def count_phrase_completes(df):
    '''
        Counts Number of times phrase complete was used
        Phrase complete happens when a user presses space or tab, the character count changes and the word count also changes 
    '''
    phrase_complete_count = 0
    itemLength = len(df['key'])
    for current_idx in range(1, itemLength):
            
        if df['key'][current_idx] in [9, 39]:
            if len(df['userText'][current_idx]) > len(df['userText'][current_idx-1]):
                if (len(df['userText'][current_idx].split()) > len(df['userText'][current_idx-1].split())):
                    phrase_complete_count += 1
               
               
    return phrase_complete_count

def count_tab_key_pressed(df):
    '''
        Counts Number of times API for phrase or word complete was used
    '''
    tab_key_count = 0
    itemLength = len(df['key'])
    for current_idx in range(1, itemLength):
        if df['key'][current_idx] in [9, 39]:
                if len(df['userText'][current_idx]) > len(df['userText'][current_idx-1]):
                    if (len(df['userText'][current_idx].split()) >= len(df['userText'][current_idx-1].split())):
                        tab_key_count += 1
    
    return tab_key_count
    
def count_words_accepted_per_phrase_complete(df):
    '''
        Counts the number of words that were accepted each time phrase complete was used
    '''
    words_accepted_per_phrase_complete = []
    current_idx=1
    itemLength = len(df['key'])
    while current_idx < itemLength:
         
        if df['key'][current_idx] in [9, 39]:
            if len(df['userText'][current_idx]) > len(df['userText'][current_idx-1]):
                if (len(df['userText'][current_idx].split()) > len(df['userText'][current_idx-1].split())):
                    phrase_complete_word_count = 1
                    next_idx = current_idx + 1
                    while(df['key'][next_idx] in [9, 39]):
                        phrase_complete_word_count+=1
                        next_idx+=1
                    words_accepted_per_phrase_complete.append(phrase_complete_word_count)
                    current_idx = next_idx
        current_idx+=1
                        
    
    return words_accepted_per_phrase_complete
                        
                    
def count_number_of_characters_deleted(df):
    
    '''
        Counts the number of deleted characters
    '''
    
    counts = Counter(df['key'])
    return counts[8]

def count_number_of_words_deleted(df):
    '''
        Counts the number of deleted words. Everytime delete key is pressed
    '''
    count_words_deleted = 0
    current_idx=0
    itemLength = len(df['key'])
    while current_idx < itemLength:
        
        if df['key'][current_idx] == 8:
            
            next_idx = current_idx + 1
            while(df['key'][next_idx] == 8):
                next_idx+=1
            count_words_deleted += (len(df['userText'][current_idx].split()) - len(df['userText'][next_idx].split()))
#             print(np.setdiff1d(df['text'][i].split(), df['text'][j].split()))
            current_idx = next_idx
        current_idx+=1
            
    return count_words_deleted
            
    
                
    

In [7]:
for file in files:
	df = json.load(open(file))
	print(file, len(df['key']), len(df['userText']))

/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-07-14 18:05:40.168753 Roma.txt 2019 2020
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-06-14 05:14:02.642773 Aryamaan with.txt 2548 2549
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-06-11 17:17:37.994480 Priya.txt 2045 2046
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-07-13 17:03:14.262589 Aditya.txt 1833 1834
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-07-19 18:42:27.634692 Soham.txt 851 852
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-07-13 13:23:03.856784 Aditav.txt 4137 4138
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-06-13 16:01:18.830920 Tejaswini with.txt 994 995
/home/advaitmb/notebooks/projects/PreDiction2.0/backend/api/logs/Final/log2021-07-20 04:04:43.635644 Rajat.txt 2685 2686
/home/advaitmb/not

In [8]:
def compute_stats(files):
	df_dict = {
	'file_name': [],
	'word_completes': [],
	'phrase_completes': [],
	'tab_pressed': [],
	'words_accepted_per_phrase_complete': [],
	'words_accepted_per_phrase_complete_mean': [],
	'characters_deleted': [],
	'words_deleted': []
	}
	for file in files:
		df = json.load(open(file))
		difference = len(df['userText']) - len(df['key'])
		for _ in range(difference):
			df['key'].insert(0, -1)
		print(len(df['userText']) - len(df['key']))	
		df_dict['word_completes'].append(count_word_completes(df))
		df_dict['phrase_completes'].append(count_phrase_completes(df))
		df_dict['tab_pressed'].append(count_tab_key_pressed(df))
		df_dict['words_accepted_per_phrase_complete'].append(count_words_accepted_per_phrase_complete(df))
		df_dict['words_accepted_per_phrase_complete_mean'].append(mean(count_words_accepted_per_phrase_complete(df)))
		df_dict['characters_deleted'].append(count_number_of_characters_deleted(df))
		# df_dict['words_deleted'].append(count_number_of_words_deleted(df))
	return pd.DataFrame(df_dict)
        

In [11]:
compute_stats(files)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


ValueError: arrays must all be same length

In [21]:
# Complete suggestion acceptance
# complete suggestion acceptance will happen when the length of suggestionText is equal to the number of times tab key is pressed 

def completeSuggestionCount(df):
	completeAcceptanceCount = 0
	i = 0
	while i < len(df['key']):
		if df['key'][i] == 9:
			suggestionArray = df['suggestionText'][i].strip().split(' ')
			suggestionLength = len(suggestionArray)
			print(suggestionLength)
			countList = df['key'][i:i+suggestionLength]
			print(countList)
			if all(x==countList[0] for x in countList):
				completeAcceptanceCount += 1
			else:
				for item in suggestionArray:
					if '.' in item: 
						completeAcceptanceCount += 1
						break	
			i+=suggestionLength
		else:
			i+=1
	return completeAcceptanceCount



# Words accepted per phrase complete

def countWordsAcceptedPerPhraseComplete(df):
	'''
	Counts the number of words that were accepted each time phrase complete was used
	'''
	i = 0
	counter = 0
	counterRegister = []
	while i < len(df['key']): 
		if df['key'][i] == 9:
			while df['key'][i] == 9:
				counter +=1 
				i+=1
			counterRegister.append(counter)
		counter = 0
		i+=1
	return counterRegister
	



# acceptance over time

def acceptanceOverTime(df):
	acceptanceRegister = []
	for i in df['key']:
		if i == 9:
			acceptanceRegister.append(1)
		else:
			acceptanceRegister.append(0)
	return acceptanceRegister


# Acceptance after fullstops 

def acceptanceAfterFullStop(df):
	counter = 0
	for i in range(1, len(df['key'])):
		if df['key'][i-1] == 91 and df['key'][i] == 9:
			counter+=1 
	return counter

def averageLengthOfSentences(df):
	userTextArray = df['userText'][-1].split('.')
	return sum([ len(i) for i in  userTextArray])/len(userTextArray)

def wordCount(df):

	return len(df['userText'][-1].split(' ')) 

			

In [25]:
quantDict = {
	'fileName': [],
	'completeSuggestionCount': [],
	'countWordsAcceptedPerPhraseComplete': [],
	'acceptanceOverTime': [],
	'acceptanceAfterFullStop': [],
	'averageLengthOfSentences': [],
	'wordCount': []
}

In [26]:
for file in files:
	df = json.load(open(file))
	quantDict['completeSuggestionCount'].append(completeSuggestionCount(df))
	quantDict['fileName'].append(file)
	quantDict['countWordsAcceptedPerPhraseComplete'].append(countWordsAcceptedPerPhraseComplete(df))
	quantDict['acceptanceOverTime'].append(acceptanceOverTime(df))
	quantDict['acceptanceAfterFullStop'].append(acceptanceAfterFullStop(df))
	quantDict['averageLengthOfSentences'].append(averageLengthOfSentences(df))
	quantDict['wordCount'].append(wordCount(df))

8
[9, 32, 67, 79, 77, 69, 68, 89]
9
[9, 32, 79, 70, 32, 84, 72, 69, 32]
8
[9, 32, 75, 73, 68, 32, 65, 78]
9
[9, 9, 9, 9, 9, 9, 9, 190, 8]
10
[9, 9, 32, 67, 79, 78, 67, 69, 65, 8]
9
[9, 9, 32, 84, 69, 72, 8, 8, 72]
8
[9, 9, 32, 16, 65, 32, 77, 85]
8
[9, 32, 8, 39, 32, 73, 78, 32]
10
[9, 32, 84, 72, 73, 83, 32, 77, 79, 86]
7
[9, 9, 9, 9, 9, 9, 9]
1
[9]
9
[9, 9, 9, 9, 9, 9, 9, 9, 9]
9
[9, 9, 9, 9, 9, 32, 75, 69, 69]
9
[9, 17, 90, 8, 8, 39, 39, 39, 39]
10
[9, 8, 32, 65, 78, 68, 32, 8, 8, 8]
10
[9, 8, 190, 32, 16, 84, 72, 69, 32, 37]
10
[9, 8, 32, 80, 69, 82, 70, 79, 82, 77]
10
[9, 32, 67, 65, 78, 78, 79, 84, 32, 66]
10
[9, 32, 87, 73, 84, 72, 32, 79, 78, 69]
8
[9, 9, 9, 9, 9, 9, 9, 9]
9
[9, 9, 9, 9, 9, 9, 9, 9, 9]
9
[9, 9, 9, 9, 9, 9, 9, 9, 9]
1
[9]
8
[9, 9, 9, 9, 9, 9, 9, 9]
9
[9, 9, 9, 9, 9, 9, 9, 9, 9]
9
[9, 9, 9, 9, 8, 32, 65, 78, 68]
8
[9, 9, 9, 9, 9, 9, 9, 8]
18
[9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 66, 85, 76, 76, 69, 84]
24
[9, 9, 9, 9, 8, 32, 66, 85, 84, 32, 68, 73, 82, 84, 32, 66,